In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sps
import warnings
from math import isclose
sns.set()

plt.rc('font', size=30)
plt.rc('axes', titlesize=30)
plt.rc('axes', labelsize=30)
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)
plt.rc('legend', fontsize=30)
plt.rc('figure', titlesize=30)

from sklearn.base import BaseEstimator
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.covariance import MinCovDet
import cvxpy as cvx

In [39]:
class NewPortfolioOptimizer(BaseEstimator):
    '''
    Класс, оптимизирующий ковариационную матрицу Марковица нужным способом, находящий оптимальные веса 
    для различных компонент портфеля, считающий по полученным весам прибыль и метрики. 
    Интерфейс класса схож с интерфейсом класса со sklearn, при этом PortfolioOptimizer является наследником
    класса BaseEstimator, что позволяет использовать написанный класс в оптимизаторах гиперпараметров.
    '''
    
    def __init__(self, R=2e-3, R_quantille=None, is_PCA=False, is_kenrel_PCA=False, is_MCD=False, n_top_companies=20,
                 n_components=1, kernel='poly', kernelgamma=None, kerneldegree=3, kernelcoef0=1, kernelparams=None, 
                 size_of_window=None, risk_free_return=1, smooth_function=None, period_change_portfolio=360,
                 period_for_pred=120, metric='sharp', VAR_quantile=0.05, verbosity=False, threshold=1e-5):
        '''
        Функция инициализации.
        
        Параметры:
        ----------
            1) Параметры портфеля:
                R : float, default=2e-3. Ожидаемый доход портфеля (за один блок). Если R_quantille != None, 
                    то используется R_quantile.
                R_quantille : float/None, default=none. Квантиль ожидаемого дохода портфеля среди топ-компаний.
                              Должен быть в промежутке [0, 1]. Если None, то используется R.
            **********
            
            2) Параметры оптимизаторов матрицы ковариации:
                +++ Либо один, либо ни одного из следующих трех полей может быть True +++
                    is_PCA : bool, default=False. Надо ли использовать PCA;
                    is_kernel_PCA : bool, default=False. Надо ли использовать KernelPCA;
                    is_MCD : bool, default=False. Надо ли использовать MinCovDet.
                
                +++ Параметры PCA/KernelPCA +++
                    n_components : int, default=1. Сколько компонент брать в PCA/KernelPCA. 
                                   Используется, если is_PCA или is_kernel_PCA == True;
                    kernel : str, default='poly'. Какое ядро использовать в KernelPCA. 
                                  Используется, если is_kernel_PCA == True;
                    kernelgamma : int/None, default=None. Параметр gamma в KernelPCA.
                                  Используется, если is_kernel_PCA == True;
                    kerneldegree : int, default=3. Степень полиномиального ядра в KernelPCA.
                                   Используется, если is_kernel_PCA == True и kernel=='poly';
                    kernelcoef0 : float, dedault=1. Параметр coef0 в KernelPCA.
                                  Используется, если is_kernel_PCA == True;
                    kernelparams : dict/None, default=None. Параметр kernel_params в KernelPCA.
                                   Пока нигде не используется, но может полезен при работе с другими ядрами.
            **********
            
            3) Параметры окна и периодов:
                size_of_window : int/None, default=None. Размер окна (в строчках), по которому используем данные.
                period_for_pred : int/None, default=120. Период, за который считаем метрику.
                period_change_portfolio : int/None, default=360. Период, с которым меняем портфель.
            **********
            
            4) Параметры подсчета метрики:
                metric : str, default='sharp'. Если 'sortino', то используем в качестве метрики коэффициент Сортино,
                         если 'sharp', то используем в качестве метрики коэффициент Шарпа, 
                         если 'VAR', то используем в качестве метрики Value at Risk.
            **********
            
            5) Параметры сглаживания:
                smooth_function : function/None, default=None. Функция сглаживания.
            **********
            
            6) Параметры логов:
                verbosity : bool, default=False. Стоит ли выводить логи (пока что выводятся только логи CVXPY).
            **********
        ----------
        
        Возвращает: ---
        
        '''
        
        self.R = R
        self.R_quantille = R_quantille
        self.is_PCA = is_PCA
        self.is_kenrel_PCA = is_kenrel_PCA
        self.is_MCD = is_MCD
        self.n_components = n_components
        self.n_top_companies = n_top_companies
        self.kernel = kernel
        self.kernelgamma = kernelgamma
        self.kerneldegree = kerneldegree
        self.kernelcoef0 = kernelcoef0
        self.kernelparams = kernelparams        
        self.size_of_window = size_of_window            
        self.risk_free_return = risk_free_return
        self.smooth_function = smooth_function
        self.period_for_pred = period_for_pred
        self.metric = metric
        self.VAR_quantile = VAR_quantile
        self.period_change_portfolio = period_change_portfolio
        self.verbosity = verbosity
        self.w_ = []
        self.arg_top_returns_ = []
        self.threshold = threshold
    
    def _decrease_risk(self):
        '''
        Функция решения задачи оптимизации портфельной теории Марковица.
        
        Параметры: ---
        
        Возвращает:
        ----------
            opt_weights : np.array(float). Оптимальные веса по портфельной теории Марковица.
        
        '''
        
        p = len(self.optimized_mu_)
        w = cvx.Variable(p)
        obj = cvx.Minimize(1/2 * cvx.quad_form(w, self.optimized_Sigma_))

        equal_constraints_1 = [self.optimized_mu_.T @ w == self.R]
        equal_constraints_2 = [np.ones(p) @ w == 1]
        eyes = np.eye(p)
        nonequal_constraints = [eye @ w >= 0 for eye in eyes]
        constraints = equal_constraints_1 + equal_constraints_2 + nonequal_constraints
        
        if np.all(np.linalg.eigvals(self.optimized_Sigma_) > 0):
            try:
                problem = cvx.Problem(obj, constraints=constraints)
                result = problem.solve(verbose=self.verbosity, solver='SCS')
            except:
                warnings.warn('''SolverError: solver can't solve this task. 
                Trying to solve with another solver''')
                problem = cvx.Problem(obj, constraints=constraints)
                result = problem.solve(verbose=self.verbosity, solver='CVXOPT')  
            
        else:
            warnings.warn("Covariance matrix is not a positive-definite")
            problem = cvx.Problem(obj, constraints=constraints)
            result = problem.solve(verbose=self.verbosity, solver='CVXOPT')       
            
        opt_weights = w.value
        return opt_weights
    
    
    def _count_portfolio_return(self, block, t=0):
        '''
        Функция подсчета ретёрна портфеля.
        
        Параметры: 
        ----------
            block : pd.DataFrame. Блок, за который считаем return.
        ----------
        
        Возвращает:
        ----------
            portfolio_return: pd.Series. Таблица из 2 столбцов: индексом служит время,
                              во втором столбце кумулятивные ретёрны, соответствующие этому времени.
        
        '''
        
        portfolio_return = None
        if self.w_[t] is None:
            self.w_[t] = np.ones(shape=(self.n_top_companies)) / self.n_top_companies
            
        for w, col in zip(self.w_[t], self.arg_top_returns_[t]):
            one_company_return = w * (block[col] + 1).cumprod()
            if portfolio_return is None:
                portfolio_return = one_company_return
            else:
                portfolio_return += one_company_return
        
        return portfolio_return           
        
    def refit(self, X_train, Y_train=None):
        self.w_ = []
        self.arg_top_returns_ = []
        self.fit(X_train)
    
        
    def fit(self, X_train, Y_train=None):
        '''
        Функция для обучения модели.
        1) Выбирает последние элементы по размеру окна.
        2) Вычисление вектора ожидаемой доходности и ковариационной матрицы доходностей.
        3) Оптимизирует коваривационную матрицу, если это требуется.
        4) Решает задачу минимизации ковариационной матрицы портфеля. Находит оптимальные веса активов.
        
        Параметры: 
        ----------
            X_train : pd.DataFrame. Датасет, на котором обучаемся.
            Y_train : pd.Series/None, default=None. 
            Параметр, который не используется. Нужен, чтоб некоторые функции принимали реализуемый класс.
        ----------
        
        Возвращает:
        ----------
            self : PortfolioOptimizer class. Обученный объект класса.
        '''
        
        if self.n_components > self.n_top_companies:
            warnings.warn('''self.n_components > self.n_top_companies;
            self.n_components is set equal to self.n_top_companies''')
            self.n_components = self.n_top_companies
        
        if self.size_of_window is not None and self.period_change_portfolio is not None \
            and self.size_of_window < self.period_change_portfolio:
            warnings.warn('''size_of_window less than period_change_portfolio;
                          size_of_window was set the same as period_change_portfolio''')
            self.size_of_window = self.period_change_portfolio
        
        self.X_train_ = X_train
        
        # выбор последних значений по размеру окна
        if self.size_of_window is not None:
            self.X_train_window_ = X_train.iloc[-self.size_of_window:]
            
        else:
            self.X_train_window_ =  X_train

        # вычисление вектора ожидаемой доходности и ковариационной матрицы доходностей
        mean = self.X_train_window_[self.X_train_window_.columns].mean()
        self.top_returns_ = self.X_train_window_[mean.sort_values(ascending=False)[:self.n_top_companies].index]
        self.mu_ = mean.sort_values(ascending=False)[:self.n_top_companies].to_numpy()

        self.Sigma_ = (self.top_returns_[self.top_returns_.columns]).cov()    
                        
        # оптимизация ковариационной матрицы
        if self.is_PCA or self.is_kenrel_PCA:
            if self.is_PCA:
                pca = PCA(n_components=min(self.n_components, self.n_top_companies))
                pca.fit(self.top_returns_.cov())
                self.components_ = pca.components_
            else:
                kpca = KernelPCA(n_components=min(self.n_components, self.n_top_companies), 
                kernel=self.kernel, degree=self.kerneldegree, gamma=self.kernelgamma, 
                coef0=self.kernelcoef0, kernel_params=self.kernelparams)
                kpca.fit(self.top_returns_.cov())
                self.components_ = kpca.eigenvectors_.T
            self.components_ = pd.DataFrame(self.components_)
            self.components_[self.components_ < 0] = 0
            self.components_ = self.components_.div(self.components_.sum(axis=1), axis=0)
            assert isclose(self.components_.sum(axis='columns').iloc[0], 1, rel_tol=1e-3), \
            f'sum of weights of the first component {self.components_.sum(axis=0).iloc[0]} is not close with 1'
            new_data = self.top_returns_.to_numpy() @ self.components_.to_numpy().T
            self.optimized_data_ = pd.DataFrame(columns=np.linspace(1, new_data.shape[1], new_data.shape[1], dtype=int), 
                                               index=self.top_returns_.index, data=new_data)
            self.optimized_mu_ = self.optimized_data_[self.optimized_data_.columns].mean().to_numpy()
            self.optimized_Sigma_ = self.optimized_data_[self.optimized_data_.columns].cov()
        
            if not np.all(np.linalg.eigvals(self.optimized_Sigma_) > 0) & \
                    np.all(self.optimized_Sigma_ == self.optimized_Sigma_.T):
                warnings.warn("Covariance matrix after PCA/KernelPCA is not a positive-definite or symmetric")
                self.optimized_data_ = self.X_train_window_
                self.optimized_Sigma_ = self.Sigma_
                self.optimized_mu_ = self.mu_
                
        elif self.is_MCD:
            mcd = MinCovDet()
            mcd.fit(self.top_returns_.cov())
            self.optimized_mu_ = self.mu_
            self.optimized_Sigma_ = mcd.covariance_

        else:
            self.optimized_mu_ = self.mu_
            self.optimized_Sigma_ = self.Sigma_

        
        if not np.all(np.linalg.eigvals(self.optimized_Sigma_) > 0) & \
                    np.all(self.optimized_Sigma_ == self.optimized_Sigma_.T):
            warnings.warn("Covariance matrix is not a positive-definite or symmetric")

        if self.R_quantille is not None:
            self.R = np.quantile(self.optimized_mu_, self.R_quantille)
        
        if np.sort(self.optimized_mu_)[0] > self.R:
            warnings.warn(f'R less than the least of expecting returns :')
            warnings.warn(f'R = {self.R}, the least of expecting returns = {self.optimized_mu_[0]}')
            self.R = min(np.quantile(self.optimized_mu_, 0.75), self.optimized_mu_[-1])
        
        if np.sort(self.optimized_mu_)[-1] < self.R:
            warnings.warn(f'R greater than the greatest of expecting returns :') 
            warnings.warn(f'R = {self.R}, the greatest of expecting returns = {self.optimized_mu_[-1]}')
            self.R = min(np.quantile(self.optimized_mu_, 0.75), self.optimized_mu_[-1])
        
        
        # решение задачи минимизации ковариационной матрицы портфеля
        w = self._decrease_risk()
        if self.is_PCA or self.is_kenrel_PCA:
            if w is None:
                warnings.warn(f'w is None; w was set by default')
                w = np.ones(shape=(self.n_components)) / self.n_components
            w = self.components_.T @ w
            
        if w is None:
            warnings.warn(f'weights are None; the weights are set by default')
            w = np.ones(shape=(self.n_top_companies)) / self.n_top_companies
        w[w < self.threshold] = 0
        if not isclose(np.sum(w), 1, rel_tol=1e-3):
            warnings.warn(f'sum of weights {np.sum(w)} is not close with 1; the weights will be rebalanced')
            w = w / np.sum(w)
        self.w_.append(w)
        self.arg_top_returns_.append(self.top_returns_.columns)
        
        
        return self
        
        
    def _predict_for_period(self, period, t=0):
        '''
        Функция для предсказания ретернов за период на исторических данных.
        
        Параметры: 
        ----------
            period : pd.DataFrame. Датасет за определенный период, на котором мы предсказываем.
        ----------
        
        Возвращает:
        ----------
            block_return : np.array(float). Массив ретернов за блоки.
            all_return : pd.Series. Таблица из 2 столбцов: индексом служит время,
                         во втором столбце кумулятивные ретёрны, соответствующие этому времени.
        '''
                
        # разбиваем на блоки, считаем доходность по блокам
        n_blocks = int(-np.floor(-(len(period) / self.period_for_pred)))
        block_return = []
        all_return_2 = pd.DataFrame()
        
        for n_block in range(n_blocks):
            if n_block != n_blocks - 1:
                block = period.iloc[n_block * self.period_for_pred: (n_block + 1) * self.period_for_pred]
            else:
                block = period.iloc[n_block * self.period_for_pred:]
                
#             print(f'''начало периода: {block.index.to_numpy()[0]}''')
#             print(f'''конец периода: {block.index.to_numpy()[-1]}''')
            
            ret = self._count_portfolio_return(block, t)
            block_return.append(ret.to_numpy()[-1])
#             print(ret.to_numpy()[-1])
            if len(all_return_2) > 0:
                ret.iloc[:] = ret.iloc[:] * all_return_2.iloc[-1].to_numpy()
            all_return_2 = pd.concat([all_return_2, ret])
        
        block_return = np.array(block_return)
        all_return = self._count_portfolio_return(period, t)
#         print(all_return.head())
#         print(all_return_2.head())
#         print(all_return.tail())
#         print(all_return_2.tail())
        self.all_return = all_return
        self.all_return_2 = all_return_2
        
        return block_return, all_return
    
    def _rebalance_fit(self, period):
        '''
        Функция для обучения с учетом новых данных и перебалансировки весов.
        
        Параметры:
        ----------
            period : pd.DataFrame. Датасет за определенный период, который добавляем в обучение (новые данные).
        ----------
        
        Возвращает:
        ----------
            self : PortfolioOptimizer class. Обученный с учетом новых данных объект класса.
        '''
        
        X_train = pd.concat([self.X_train_, period])
        self.fit(X_train)
        return self
    
    
    def predict(self, X_test, Y_test=None):
        '''
        Функция для предсказания ретернов на исторических данных.
        
        Параметры: 
        ----------
            X_test : pd.DataFrame. Тестовый датасет.
            Y_test : pd.Series/None, default=None. 
            Параметр, который не используется. Нужен, чтоб некоторые функции принимали реализуемый класс.
        ----------
        
        Возвращает:
        ----------
            block_return : np.array(float). Массив ретернов за блоки.
            all_return : pd.Series. Таблица из 2 столбцов: индексом служит время,
                         во втором столбце кумулятивные ретёрны, соответствующие этому времени.
        '''
        
        old_X_train = self.X_train_
        old_w = self.w_
        old_arg_top_returns = self.arg_top_returns_
        
        if self.period_change_portfolio is None:
            self.period_change_portfolio = len(X_test) + 1
        
        block_return = np.array([])
        all_return = pd.Series(dtype='float64')
        
        # разбиваем данные на периоды
        num_periods = 1 + int(len(X_test) / self.period_change_portfolio)
        for t in range(num_periods):
            if t != num_periods - 1:
                period = X_test.iloc[t * self.period_change_portfolio: (t + 1) * self.period_change_portfolio]
            else:
                period = X_test.iloc[t * self.period_change_portfolio:]
            
            # сделаем предсказание на период
            period_return = self._predict_for_period(period, t)
            block_return = np.append(block_return, period_return[0])
            if len(all_return) > 0:
                period_return[1].iloc[:] = np.array(period_return[1].iloc[:]) * all_return[-1]
            all_return = pd.concat([all_return, period_return[1]])
            
            # сделаем фит с учетом новых параметров
            if t != num_periods - 1:
                self._rebalance_fit(period)
            
        self.X_train_ = old_X_train
        self.w_ = old_w
        self.arg_top_returns_ = old_arg_top_returns
            
        return block_return, all_return     
    
    
    def score(self, X_test, Y_test=None, how=None, was_predicted=False):
        '''
        Функция для подсчета метрики.
        
        Параметры: 
        ----------
            X_test : pd.DataFrame. Тестовый датасет.
            Y_test : pd.Series/None, default=None. 
            Параметр, который не используется. Нужен, чтоб некоторые функции принимали реализуемый класс.
            how : str/None, default=None. Какую метрику использовать. Если 'sortino', 
            то используется коэффициент Сортино, если None или 'sharp', то используется коэффициет Шарпа.
        ----------
        
        Возвращает:
        ----------
            sc : нужный коэффициент (Шарпа или Сортино)
        '''
        
        if how is None:
            how = self.metric

        if not was_predicted:
            if len(self.w_) != 0:
                self.w_ = [self.w_[0]]
            self.block_return_ = self.predict(X_test)[0]
        if how == 'VAR':
            sc = np.quantile(self.block_return_, self.VAR_quantile) - 1
            self.score_ = sc
            return sc
        
        # высчитываем среднюю избыточную доходность 
        avg_reduntant_return = np.mean(self.block_return_ - self.risk_free_return)
        
        # высчитываем стандартное отклонение портфеля
        if how == 'sharp':
            std_deviation_portfolio = np.std(self.block_return_ - self.risk_free_return, ddof=1)
        else:
            std_deviation_portfolio = np.std(np.minimum(0, self.block_return_ - \
                                                        self.risk_free_return), ddof=1)
            
        
        # считаем нужный коэффициент
        sc = avg_reduntant_return / std_deviation_portfolio
        self.score_ = sc
        
        return sc
    
    
#     def _count_all_portfolio_return(self, X_test):
#         '''
#         Функция подсчета ретёрна портфеля на всем тесте с учетом ребалансировок.
        
#         Параметры: 
#         ----------
#             block : pd.DataFrame. Блок, за который считаем return.
#         ----------
        
#         Возвращает:
#         ----------
#             port_series: pd.Series. Таблица из 2 столбцов: индексом служит время,
#                               во втором столбце кумулятивные ретёрны, соответствующие этому времени.
        
#         '''
        
#         port_series = pd.DataFrame()
#         for t, weights in enumerate(self.w_):
#             portfolio_return = 0
#             if t < (len(self.w_) - 1):
#                 block = X_test[t * self.period_change_portfolio:(t + 1) * self.period_change_portfolio]
#             else:
#                 block = X_test[t * self.period_change_portfolio:]
                
#             for w, col in zip(weights, self.arg_top_returns_[t]):
#                 one_company_return = w * (block[col] + 1).cumprod()
#                 portfolio_return += one_company_return
            
#             if len(port_series) > 0:
#                 portfolio_return.iloc[:] = portfolio_return.iloc[:] * port_series.iloc[-1].to_numpy()
#             port_series = pd.concat([port_series, portfolio_return])
        
#         return port_series